In [1]:
import pandas as pd
df = pd.read_csv('../results/mai_doc2vec_sim.csv')
df

In [2]:
import spacy
nlp = spacy.load('ja_ginza')
for p in nlp.pipeline:
    print(p)

In [3]:
import textdistance
# text_pair = df.sort_values('SIM', ascending=False).iloc[203][['T2_Mai', 'T2_Maisho', 'SIM']].values.flatten()
text_pair = df.sample(1)[['T2_Mai', 'T2_Maisho', 'SIM']].values.flatten()
print(text_pair)
print('longest common subsequence：', textdistance.lcsseq(text_pair[0], text_pair[1]))
print('distance: ', textdistance.lcsseq.distance(text_pair[0], text_pair[1]))
print('normalized distance: ', textdistance.lcsseq.normalized_distance(text_pair[0], text_pair[1]))
print('similarity: ', textdistance.lcsseq.similarity(text_pair[0], text_pair[1]))
print('normalized similarity: ', textdistance.lcsseq.normalized_similarity(text_pair[0], text_pair[1]))

In [4]:
for i in range(2):
    print('-------------------------------------------------')
    doc = nlp(text_pair[i])
    for s in doc.sents:
        print(s)

    from spacy import displacy
    displacy.render(doc, style="ent")

In [5]:
import stanfordnlp
nlp = stanfordnlp.Pipeline(lang="ja", treebank="ja_gsd")
for i in range(2):
    print('-------------------------------------------------')
    doc = nlp(text_pair[i])
    for sen in doc.sentences:
        print(' '.join(word.text for word in sen.words))

In [6]:
%%time
import numpy as np
@np.vectorize
def lcs(t1, t2):
    return textdistance.lcsseq.normalized_similarity(t1, t2)

df['lcs_sim'] = lcs(df['T2_Mai'], df['T2_Maisho'])

In [0]:
df.sort_values('SIM', ascending=False)['lcs_sim'].plot.hist(bins=1000)